### A simple Decision tree built from scratch, using Iris dataset as an example
In this repo, I implement a simple decision tree from scratch (for knowledge review) as well as using sklearn library. I referred to the materials from this blog https://towardsdatascience.com/decision-tree-from-scratch-in-python-46e99dfea775

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv('datasets/Iris.csv')
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
df.describe()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,75.500000,5.843333,3.054000,3.758667,1.198667
std,43.445368,0.828066,0.433594,1.764420,0.763161
min,1.000000,4.300000,2.000000,1.000000,0.100000
25%,38.250000,5.100000,2.800000,1.600000,0.300000
50%,75.500000,5.800000,3.000000,4.350000,1.300000
75%,112.750000,6.400000,3.300000,5.100000,1.800000
max,150.000000,7.900000,4.400000,6.900000,2.500000


In [4]:
le = LabelEncoder()
df['Species_encoded'] = le.fit_transform(df['Species'])

In [5]:
df.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,Species_encoded
0,1,5.1,3.5,1.4,0.2,Iris-setosa,0
1,2,4.9,3.0,1.4,0.2,Iris-setosa,0
2,3,4.7,3.2,1.3,0.2,Iris-setosa,0
3,4,4.6,3.1,1.5,0.2,Iris-setosa,0
4,5,5.0,3.6,1.4,0.2,Iris-setosa,0


In [6]:
train_trg = df['Species_encoded']
train_inp = df[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]

In [7]:
train_inp, test_inp, train_trg, test_trg = train_test_split(train_inp, train_trg, test_size=130, stratify=train_trg)

In [8]:
print('Train input size: {}'.format(train_inp.shape))
print('Train target size: {}'.format(train_trg.shape))
print('Test input size: {}'.format(test_inp.shape))
print('Test target size: {}'.format(test_trg.shape))

Train input size: (20, 4)
Train target size: (20,)
Test input size: (130, 4)
Test target size: (130,)


#### 1. Build from scratch

In [9]:
class Node(object):
    def __init__(self):
        self.best_feature = None
        self.best_threshold = None
        self.gini = None
        self.predicted_class = None
        self.left = None
        self.right = None
    
    def __str__(self):
        return ('Best feature: %s, best threshold: %s, gini: %s, class: %s'%(str(self.best_feature), str(self.best_threshold), str(self.gini), str(self.predicted_class)))
    
class DecisionTree(object):
    def __init__(self, max_depth):
        self.root = Node()
        self.max_depth = max_depth
        
    def fit(self, inp, trg):
        self.features = inp.columns
        if type(inp) == pd.core.frame.DataFrame:
            inp = inp.to_numpy()
        if type(trg) == pd.core.frame.DataFrame:
            trg = trg.to_numpy()
        self.build_tree(self.root, inp, trg, 0)
        
        
    def build_tree(self, node, inp, trg, depth):
        m = inp.shape[0]
        classes = set(trg)
        num_cls = dict(zip(classes, [sum(trg==c) for c in classes]))
        node.gini = 1 - sum([(num_cls[c]/m)**2 for c in classes])
        best_gini = node.gini
        node.predicted_class = max(num_cls, key=lambda c: num_cls[c])
        
        # find best split
        for feature in range(inp.shape[1]):
            num_cls_left = dict(zip(classes, [0]*len(classes)))
            num_cls_right = num_cls.copy()
            x, y = zip(*sorted(zip(inp[:, feature], trg)))
            for idx in range(1, m):
                cls = y[idx]
                num_cls_left[cls] += 1
                num_cls_right[cls] -= 1
                gini_left = 1 - sum([(num_cls_left[c]/idx)**2 for c in classes])
                gini_right = 1 - sum([(num_cls_right[c]/(m-idx))**2 for c in classes])
                gini = gini_left * (idx/m) + gini_right * ((m-idx)/m)
                if gini < best_gini:
                    best_gini = gini
                    node.best_feature = feature
                    node.best_threshold = x[idx]
        if node.best_threshold == None or depth + 1 >= self.max_depth:
            return
        
        # recursion
        left_idx = inp[:, node.best_feature] < node.best_threshold
        inp_left, trg_left = inp[left_idx], trg[left_idx]
        inp_right, trg_right = inp[~left_idx], trg[~left_idx]
        if (len(inp_left)):
            node.left = Node()
            self.build_tree(node.left, inp_left, trg_left, depth+1)
        if (len(inp_right)):
            node.right = Node()
            self.build_tree(node.right, inp_right, trg_right, depth+1)
        
    def predict(self, test):
        if type(test) == pd.core.frame.DataFrame:
            test = test.to_numpy()
        pred = []
        for i in range(len(test)):
            node = self.root
            cls = 0
            while node != None:
                feature = node.best_feature
                threshold = node.best_threshold
                cls = node.predicted_class
                if threshold == None:
                    break
                elif test[i, feature] < threshold:
                    node = node.left
                else:
                    node = node.right
            pred.append(cls)
        return np.array(pred)

In [10]:
model1 = DecisionTree(10)
model1.fit(train_inp, train_trg)

In [11]:
train_pred_1 = model1.predict(train_inp)
print('Train accuracy: {0:.2f}%'.format(100*accuracy_score(train_pred_1, train_trg)))

Train accuracy: 100.00%


In [12]:
test_pred_1 = model1.predict(test_inp)
print('Test accuracy: {0:.2f}%'.format(100*accuracy_score(test_pred_1, test_trg)))

Test accuracy: 83.85%


#### 2. Using sklearn library

In [13]:
model2 = DecisionTreeClassifier(criterion='gini', max_depth=10)
model2.fit(train_inp, train_trg)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [14]:
train_pred_2 = model2.predict(train_inp)
print('Accuracy: %.2f%%' %(100*accuracy_score(train_pred_2, train_trg)))

Accuracy: 100.00%


In [15]:
test_pred_2 = model2.predict(test_inp)
print('Accuracy: %.2f%%' %(100*accuracy_score(test_pred_2, test_trg)))

Accuracy: 93.85%
